In [1]:
!pip install pandas
!pip install pystan==2.19.1.1
!pip install prophet
!pip install boto3

  Using cached pystan-2.19.1.1.tar.gz (16.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached cython-3.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
Using cached cython-3.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [3 lines of output]
      /tmp/pip-install-lecpnizm/pystan_bb9687f1afa74f27bac6120fca1fbc21/setup.py:61: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
        self.version = node.value.s
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
  Running setup.py clean for pystan
Failed to build pystan
ERROR: Failed to build installable wheels for some pyproject.toml based projects (pystan)
  U

In [2]:
import pandas as pd
from prophet import Prophet
import boto3
import numpy as np
import os

In [3]:
bucket = 'thedogspaw-small-forecast-data'  # <--- s3 bucket

# Connect to S3
s3 = boto3.client('s3')

def s3_get(key, local):
    s3.download_file(bucket, key, local)
    
s3_get('datasets/thedogspaw_phppos_sales.csv',         '/tmp/sales.csv')
s3_get('datasets/thedogspaw_phppos_sales_items.csv',   '/tmp/sales_items.csv')
s3_get('datasets/thedogspaw_phppos_variations_combined.csv', '/tmp/variations_combined.csv')

In [4]:
sales_df = pd.read_csv(
    '/tmp/sales.csv',
    usecols=['sale_id', 'sale_time',  'location_id'],
    parse_dates=['sale_time'],
    low_memory=False
)

sales_items_df = pd.read_csv(
    '/tmp/sales_items.csv',
    usecols=['sale_id', 'item_variation_id', 'quantity_purchased'],
    low_memory=False
)

variations_combined_df = pd.read_csv('/tmp/variations_combined.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)   # set to None to show all rows if needed

display(sales_items_df.head())

,sale_id,item_variation_id,quantity_purchased
0,1,NaN,1.0
1,2,NaN,1.0
2,3,NaN,1.0
3,4,NaN,1.0
4,5,NaN,1.0


In [5]:
print(sales_df.columns)
print(sales_items_df.columns)

Index(['sale_time', 'sale_id', 'location_id'], dtype='object')
Index(['sale_id', 'item_variation_id', 'quantity_purchased'], dtype='object')


## No preprocessing columns needed for sales.csv and sales_items.csv NaN columns to Str
## Since we dont even load them to the notebook

## We take quantity_purchased from sales_items_df We take sale_time from sales_df

In [6]:
sales_items_with_time = sales_items_df.merge(
    sales_df[['sale_id', 'sale_time']],
    on='sale_id',
    how='left'
)

print(sales_items_with_time.columns)

Index(['sale_id', 'item_variation_id', 'quantity_purchased', 'sale_time'], dtype='object')


In [7]:
# Add location and names from variations_combined_df
combined_df = sales_items_with_time.merge(
    variations_combined_df[['item_variation_id', 'location_id', 'variation_name', 'name']].drop_duplicates(),
    on='item_variation_id',
    how='left'
)

print(combined_df.columns)

Index(['sale_id', 'item_variation_id', 'quantity_purchased', 'sale_time',
       'location_id', 'variation_name', 'name'],
      dtype='object')


In [8]:
# Create a daily date column
combined_df['sale_date'] = pd.to_datetime(combined_df['sale_time']).dt.date

In [9]:
combined_df.head()

,sale_id,item_variation_id,quantity_purchased,sale_time,location_id,variation_name,name,sale_date
0,1,NaN,1.0,2012-07-13 17:22:36,NaN,NaN,NaN,2012-07-13
1,2,NaN,1.0,2012-07-13 17:26:14,NaN,NaN,NaN,2012-07-13
2,3,NaN,1.0,2012-07-13 17:31:02,NaN,NaN,NaN,2012-07-13
3,4,NaN,1.0,2012-07-13 17:48:23,NaN,NaN,NaN,2012-07-13
4,5,NaN,1.0,2012-07-13 22:13:14,NaN,NaN,NaN,2012-07-13


In [10]:
# Aggregate quantity sold per day, per variation, per location, with readable names
agg_df = combined_df.groupby(
    ['sale_date', 'item_variation_id', 'location_id', 'variation_name', 'name']
)['quantity_purchased'].sum().reset_index()

# track when does those returns happening
returns = agg_df[agg_df['quantity_purchased'] < 0]
if not returns.empty:
    display(returns.groupby(['item_variation_id', 'location_id'])['quantity_purchased'].sum().reset_index())
else:
    print("No returns in this data window.")

# exclude those negative (barang di return) value from df
agg_df = agg_df[agg_df['quantity_purchased'] >= 0]

recent_daily_var_sales = agg_df.rename(
    columns={
        'sale_date': 'date',
        'item_variation_id': 'variation_id',
        'quantity_purchased': 'y'
    }
)

,item_variation_id,location_id,quantity_purchased
0,7.0,1.0,-2.0
1,7.0,2.0,-2.0
2,17.0,1.0,-1.0
3,17.0,2.0,-1.0


In [14]:
recent_daily_var_sales.head()

,date,variation_id,location_id,variation_name,name,y
0,2024-04-27,2.0,1.0,Prices: 1.99,Bakery,2.0
1,2024-04-27,2.0,2.0,Prices: 1.99,Bakery,2.0
2,2024-04-27,7.0,1.0,Prices: 6.99,Bakery,1.0
3,2024-04-27,7.0,2.0,Prices: 6.99,Bakery,1.0
4,2024-04-27,17.0,1.0,Prices: 25.99,Bakery,1.0


In [18]:
# Ensure 'date' column is a pandas Timestamp
recent_daily_var_sales['date'] = pd.to_datetime(recent_daily_var_sales['date'])

latest_date = recent_daily_var_sales['date'].max()
cutoff_date = latest_date - pd.DateOffset(months=12)
recent_12m = recent_daily_var_sales[recent_daily_var_sales['date'] >= cutoff_date].copy()

# Add a week number column
recent_12m['sales_week'] = recent_12m['date'].dt.isocalendar().week
recent_12m['sales_year'] = recent_12m['date'].dt.isocalendar().year

# Count unique sales days and unique (year, week) combos per variation/location
history_quality = (
    recent_12m.groupby(['location_id', 'variation_id'])
    .agg(
        num_days_with_sales=('date', 'nunique'),
        num_weeks_with_sales=('sales_week', 'nunique'),
        num_years_with_sales=('sales_year', 'nunique')  # Useful for data that spans New Year
    )
    .reset_index()
)

# Merge readable names from recent_12m (drop_duplicates to avoid extra rows)
history_quality = history_quality.merge(
    recent_12m[['location_id', 'variation_id', 'variation_name', 'name']].drop_duplicates(),
    on=['location_id', 'variation_id'],
    how='left'
)

# we can require, for example, at least 20 days of sales AND sales in at least 8 different weeks
min_days = 20
min_weeks = 8

history_quality['enough_history'] = (
    (history_quality['num_days_with_sales'] >= min_days) &
    (history_quality['num_weeks_with_sales'] >= min_weeks)
)

display(history_quality.head(30))

,location_id,variation_id,num_days_with_sales,num_weeks_with_sales,num_years_with_sales,variation_name,name,enough_history
0,1.0,1.0,12,10,2,Prices: .99,Bakery,False
1,1.0,2.0,172,51,2,Prices: 1.99,Bakery,True
2,1.0,3.0,134,49,2,Prices: 2.99,Bakery,True
3,1.0,4.0,163,50,2,Prices: 3.99,Bakery,True
4,1.0,5.0,202,52,2,Prices: 4.99,Bakery,True
5,1.0,6.0,117,45,2,Prices: 5.99,Bakery,True
6,1.0,7.0,290,52,2,Prices: 6.99,Bakery,True
7,1.0,8.0,51,30,2,Prices: 7.99,Bakery,True
8,1.0,9.0,124,45,2,Prices: 13.99,Bakery,True
9,1.0,10.0,35,25,2,Prices: 19.99,Bakery,True


In [ ]:
from prophet import Prophet
from tqdm.notebook import tqdm
import numpy as np

lead_time_days = 7
z = 1.65
results = []

for _, row in tqdm(history_quality.iterrows(), total=len(history_quality)):
    loc = row['location_id']
    var = row['variation_id']
    enough = row['enough_history']
    var_sales_history = recent_daily_var_sales[
        (recent_daily_var_sales['location_id'] == loc) &
        (recent_daily_var_sales['variation_id'] == var)
    ].copy()
    var_sales_history = var_sales_history.rename(columns={'date': 'ds', 'y': 'y'})
    var_sales_history['ds'] = pd.to_datetime(var_sales_history['ds'])

    cutoff_in_loop = var_sales_history['ds'].max() - pd.DateOffset(months=12)
    var_sales_history = var_sales_history[var_sales_history['ds'] >= cutoff_in_loop]

    reorder_level = None
    replenish_level = None

    if enough and len(var_sales_history) >= 20:
        try:
            m = Prophet(daily_seasonality=True)
            m.fit(var_sales_history)
            future = m.make_future_dataframe(periods=lead_time_days)
            forecast = m.predict(future)
            lead_forecast = forecast.tail(lead_time_days)
            demand_lt = lead_forecast['yhat'].sum()
            # Estimate the standard deviation of demand over lead time, 
            # based on Prophet’s upper/lower confidence intervals (3.29 = width for 99.9% interval; adjust if want 95%, etc. Higher number = more holding stock)
            sigma_lt = (lead_forecast['yhat_upper'].sum() - lead_forecast['yhat_lower'].sum()) / 3.29
            # this is buffer
            safety_stock = z * sigma_lt
            # Set the reorder point to be demand over lead time + safety stock.
            reorder_level = int(np.round(demand_lt + safety_stock))
            # replenish = reorder + another round of lead time
            replenish_level = int(np.round(reorder_level + demand_lt))
        except Exception as e:
            last_week = var_sales_history.sort_values('ds').tail(7)
            avg_daily = last_week['y'].mean() if len(last_week) else 1
            demand_lt = avg_daily * lead_time_days
            reorder_level = int(np.round(demand_lt))
            replenish_level = int(np.round(demand_lt * 2))
    else:
        last_week = var_sales_history.sort_values('ds').tail(7)
        avg_daily = last_week['y'].mean() if len(last_week) else 1
        demand_lt = avg_daily * lead_time_days
        reorder_level = int(np.round(demand_lt))
        replenish_level = int(np.round(demand_lt * 2))

    results.append({
        'location_id': loc,
        'variation_id': var,
        'reorder_level': reorder_level,
        'replenish_level': replenish_level,
        'enough_history': enough
    })

results_df = pd.DataFrame(results)


  0%|          | 0/28 [00:00<?, ?it/s]

14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:54 - cmdstanpy - INFO - Chain [1] done processing
14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:54 - cmdstanpy - INFO - Chain [1] done processing
14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:54 - cmdstanpy - INFO - Chain [1] done processing
14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:54 - cmdstanpy - INFO - Chain [1] done processing
14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:54 - cmdstanpy - INFO - Chain [1] done processing
14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:54 - cmdstanpy - INFO - Chain [1] done processing
14:43:54 - cmdstanpy - INFO - Chain [1] start processing
14:43:55 - cmdstanpy - INFO - Chain [1] done processing
14:43:55 - cmdstanpy - INFO - Chain [1] start processing
14:43:55 - cmdstanpy - INFO - Chain [1] done processing
14:43:55 - cmdstanpy - INFO - Chain [1] start processing


In [ ]:
results_df.head()

In [ ]:
# Merge in names from agg_df
results_df = results_df.merge(
    agg_df[['item_variation_id', 'variation_name', 'name']].drop_duplicates(),
    left_on='variation_id',
    right_on='item_variation_id',
    how='left'
)


In [ ]:
# Use the same cutoff logic as before
latest_date = recent_daily_var_sales['date'].max()
cutoff_date = latest_date - pd.DateOffset(months=12)
recent_12m = recent_daily_var_sales[recent_daily_var_sales['date'] >= cutoff_date]

# Now group on recent_12m
last_sale_dates = (
    recent_12m.groupby(['location_id', 'variation_id'])['date']
    .max()
    .reset_index()
    .rename(columns={'date': 'last_sale_date'})
)


In [ ]:
# Optionally add last sale date
last_sale_dates = (
    recent_daily_var_sales.groupby(['location_id', 'variation_id'])['date']
    .max()
    .reset_index()
    .rename(columns={'date': 'last_sale_date'})
)

results_df = results_df.merge(last_sale_dates, on=['location_id', 'variation_id'], how='left')

In [ ]:
# Clean up column names for final output
clean_df = results_df[[
    'location_id', 'variation_id', 'reorder_level', 'replenish_level', 'enough_history',
    'variation_name', 'name', 'last_sale_date'
]].rename(columns={'name': 'item_name'})

display(clean_df.head())

In [ ]:
forecastable_df = clean_df[clean_df['enough_history'] == True]
display(forecastable_df.head(100))